In [1]:
import pandas as pd
import py
import yfinance as yf
from datetime import datetime 
from functools import reduce

In [334]:
path = 'File to read'
df = pd.read_csv(path,thousands=',')

In [336]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [341]:
import yfinance as yf
tickers = 'IVV'
start_date = '2016-10-01'
end_date = '2021-01-31'
freq = '1d'

In [342]:
ivv_df = yf.download(tickers, start = start_date, end = end_date, interval = freq, actions = "inline")

[*********************100%***********************]  1 of 1 completed


In [344]:
ivv_df = ivv_df.reset_index()

In [345]:
consolidate_df = ivv_df.merge(df,how='left',left_on='Date',right_on='DATE')

In [21]:
pd.set_option('display.max_rows', None)

In [347]:
consolidate_df = consolidate_df.drop(['DATE'], axis=1)

In [348]:
consolidate_df = consolidate_df[['Date','TRANSACTION','TICKER','DESCRIPTION','SHARES','CASH','Open','Dividends','Stock Splits']]

In [349]:
df = consolidate_df

In [350]:
df = df.fillna(0)

In [351]:
df['SP Price'] = df['Open']

In [354]:
df.loc[df['TRANSACTION'] != 'REINVESTMENT', 'Trans SP Share Equiv'] = df['CASH']/df['SP Price']*-1

In [134]:
df['Trans SP Share Equiv'] = df['Trans SP Share Equiv'].fillna(0)

In [357]:
df['SP Port'] = df['Trans SP Share Equiv'].cumsum()

In [358]:
#Where S&P Dividends paid, calculate additional shares and add to running total (based on 'SP Port' holdings)
df.loc[df['Dividends'] !=0, 'SP Port Div Acq'] =  df['SP Port'] * (df['Dividends'] / df['SP Price'])
df = df.fillna(0)
df['SP Port Div Acq'] = df['SP Port Div Acq'].cumsum()
#Add accumulated dividends to accumulated shares
df['Total SP Port'] = df['SP Port'] + df['SP Port Div Acq']

In [359]:
df['Port Cash'] = 0

In [360]:
df = df[['Date', 'TRANSACTION', 'TICKER', 'DESCRIPTION', 'SHARES', 'CASH', 'SP Price', 'Dividends', 'Stock Splits', 'Trans SP Share Equiv', 'SP Port','SP Port Div Acq','Total SP Port', 'Port Cash']]

In [361]:
df.rename(columns = {'CASH':'COST'}, inplace = True)

In [369]:
df.loc[df['TRANSACTION'] == 'REINVESTMENT', 'Reinvest Check'] = 1

In [371]:
# Calculate Cash deposited and Cash held

df['Port Cash'] = 0
df['Cash Deposited'] = 0
j = 0
for i in df['COST']: 
    check = df.loc[[j],['Reinvest Check']]
    if (float(i) > 0) & (check.iat[0,0] != 1):    
        df.loc[[j],['Port Cash']] = df.loc[[j],['COST']].values  
    if (float(i) < 0) & (check.iat[0,0] != 1):    
        if df.loc[[j-1],['Port Cash']].values > 0:
            df.loc[[j],['Port Cash']] = df.loc[[j],['COST']].values  
        elif (check.iat[0,0] != 1):
            df.loc[[j],['Cash Deposited']] = df.loc[[j],['COST']].values  
    if j > 0:
        df.loc[[j],['Port Cash']] = df.loc[[j],['Port Cash']].values + df.loc[[j-1],['Port Cash']].values
        df.loc[[j],['Cash Deposited']] = df.loc[[j-1],['Cash Deposited']].values + df.loc[[j],['Cash Deposited']].values
    j += 1

In [373]:
#Calculate running total value of Hypothetical S&P Portfolio
df['SP Port Value'] = df['Total SP Port'] * df['SP Price']

In [ ]:
#%% Shift to Current 

In [375]:
df_SP_Only = df

In [376]:
df = df.fillna(0)

In [378]:
# Calculate Cash deposited and Cash held

df['Actual Port Cash'] = 0
df['Actual Cash Deposited'] = 0
j = 0
for i in df['COST']: 
    if float(i) > 0:    
        df.loc[[j],['Actual Port Cash']] = df.loc[[j],['COST']].values  
    if (float(i) < 0) & (check.iat[0,0] != 1):    
        if df.loc[[j-1],['Actual Port Cash']].values > 0:
            df.loc[[j],['Actual Port Cash']] = df.loc[[j],['COST']].values  
        elif (check.iat[0,0] != 1):
            df.loc[[j],['Actual Cash Deposited']] = df.loc[[j],['COST']].values  
    if j > 0:
        df.loc[[j],['Actual Port Cash']] = df.loc[[j],['Actual Port Cash']].values + df.loc[[j-1],['Actual Port Cash']].values
        df.loc[[j],['Actual Cash Deposited']] = df.loc[[j-1],['Actual Cash Deposited']].values + df.loc[[j],['Actual Cash Deposited']].values
    j += 1

In [380]:
df.to_csv(r'C:\Users\Game\Desktop\Investments\investments analyzed.csv')